In [1]:
#!/usr/bin/env python
# coding: utf-8

In [18]:
import codecs
from datetime import datetime as dt
import pandas as pd
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly
import sys
import shutil

In [3]:
def get_gr_name(x):
    if x == 'a':
        return "(アルファ株)"
    if x == 'd':
        return "(デルタ株)"
    if x == 'd+':
        return "(デルタ＋株)"
    return "(変異株)"

In [17]:
prev = pd.read_csv("prev.csv", encoding='shift-jis', header=0)
curr = pd.read_csv("maps.csv", encoding='shift-jis', header=0)
latest_date = curr['opened'].iloc[-1].replace('/', '-')
print("latest date: {}".format(latest_date))
curr.to_excel('maps.xlsx', encoding='utf-8', index=False)

latest date: 2021-8-14


In [5]:
n = len(curr) - len(prev)
today = latest_date #= dt.now().isoformat()[:10]
head = ''

In [6]:
icons = [
    '',      #  0 NULL
    '🍴',    #  1 飲食店
    '🍻',    #  2 キャバクラ
    '🏢',    #  3 事業所
    '👨‍🦽',  #  4 福祉施設
    '🏥',    #  5 医療機関
    '🏫',    #  6 小学校
    '🏫',    #  7 中学校
    '🏫',    #  8 高校
    '🏫',    #  9 大学
    '🏫',    # 10 専門学校
    '👮‍♂️', # 11 警察
    '🛡',    # 12 自衛隊
    '🏫',    # 13 教育機関
    '👨‍🚒',  # 14 消防署
    '️⚽️',   # 15 運動施設
    '🧒',    # 16 保育園/幼稚園
    '🎤',   # 17 カラオケ
    '⛹🏻‍♂️',   # 18 体育館
    '🏭',   # 19 工場
    '🏊🏼‍♀️',   # 20 水泳
    '🥋',   # 21 柔道空手
    '🏨',   # 22 ホテル
    '☎️',   # 23 コールセンター
    '🚕',   # 24 タクシー
    '🚌',   # 25 バス
    '🚗',   # 26 自家用車
    '💇‍♀️',# 27 理美容室
    '🛳️', # 28 船
    '📚', # 29 図書館 
    '🚑', # 30 救急車 
    '🏧', # 31 金融機関 
    '🧑‍🍳', # 32 調理室
    '🏡', # 33 個人宅 
    '🚊', # 34 電車
    '🐟', # 35 生鮮市場
]

In [7]:
def get_prev_cases_edu(df, name):
    results = []
    total = 0
    ico = "🧑‍🎓"
    if name.endswith("小学校"):
        ico = "🎒"
    ret = "{}{}では過去に".format(ico, name)
    prev_cases = curr[
        (curr['name'] == name) & 
        (curr['opened'] != curr['opened'].iloc[-1])]
    if len(prev_cases) == 0:
        return results
    for k, v in prev_cases.iterrows():
        new_str = "{}の{}名、".format(v['opened'], v['patients'])
        if len(ret) + len(new_str) < 160:
            ret += new_str
        else:
            results.append(ret)
            ret = new_str
        total += v['patients']
    ret = ret.rstrip('、')
    new_str = "の{}回(計{}名)の感染が確認されています。".format(len(prev_cases), total)
    if len(ret) + len(new_str) < 160:
        results.append(ret + new_str)
    else:
        results.append(ret)
        results.append(new_str)
    #print(results)
    return results

In [8]:
msg = []
spr = []
sch = []
if n > 0:
    head = "{}、道内で {} 件の新規感染報告が発生しています。".format(today, n)
else:
    head = "{}、北海道の新型コロナ感染者の報告状況です。".format(today)
for k, v in curr.iterrows():
    s = ""
    name = ""
    if k < len(prev):
        diff = curr.at[k, 'patients'] - prev.at[k, 'patients']
        if diff > 0:
            name = curr.at[k, 'name']
            s = "{}、{} {}名[+{}名]".format(
                curr.at[k, 'state'], curr.at[k, 'name'],
                curr.at[k, 'patients'], diff)
    else:
        s = "{}、{} {}名🆕".format(
            curr.at[k, 'state'], curr.at[k, 'name'], curr.at[k, 'patients'])
        name = curr.at[k, 'name']
    if s != "":
        if name.endswith("学校"):
            prev_cases = get_prev_cases_edu(curr, name)
            if len(prev_cases) > 0:
                sch.extend(prev_cases)
        if not pd.isnull(curr.at[k, 'ACH']) and curr.at[k, 'ACH'] != '':
            s += get_gr_name(curr.at[k, 'ACH'])
        if s.startswith('札幌市、'):
            s = s.replace("札幌市、", "").replace("札幌市立", "").replace('市立','')
            s = icons[curr.at[k, 'icon']] + s
            spr.append(s)
        else:
            s = icons[curr.at[k, 'icon']] + s
            msg.append(s)

In [9]:
if len(msg) > 0 or len(spr) > 0:
    f = codecs.open("diffs.txt", "w", encoding="utf-8")
    #f.write(head + "\n")
    if len(spr) > 0:
        f.write("■札幌市\n")
        for s in sorted(spr):
            f.write("{}\n".format(s))
    if len(msg) > 0:
        f.write("■札幌以外\n")
        for m in sorted(msg):
            f.write("{}\n".format(m))
    f.write("\n皆様の一日も早いご回復をお祈り申し上げます。\n")
    if len(sch) > 0:
        f.write("🏫学校関連\n")
        for s in sorted(sch):
            f.write("{}\n".format(s))
    f.close()

In [10]:
print(head)
print("Finished, dounai:{}, sapporo:{}".format(len(msg), len(spr)))

2021-8-14、道内で 10 件の新規感染報告が発生しています。
Finished, dounai:7, sapporo:13


In [11]:
#msg

In [12]:
tw = get_twitter()

In [13]:
if "ipy" in sys.argv[0]:
    exit()

In [ ]:
if len(msg) == 0 and len(spr) == 0:
    exit()

In [15]:
u = 'https://geneasyura.github.io/cov19-hm/hokkaido-map.html'
c = 'https://raw.githubusercontent.com/geneasyura/cov19-hm/master/csv/maps.xlsx'
heads = "{}\n地図: {}\nExcel: {}\nCSV: {}".format(head, u, c, c.replace('xlsx', 'csv'))
heads

'2021-8-14、道内で 10 件の新規感染報告が発生しています。\n地図: https://geneasyura.github.io/cov19-hm/hokkaido-map.html\nExcel: https://raw.githubusercontent.com/geneasyura/cov19-hm/master/csv/maps.xlsx\nCSV: https://raw.githubusercontent.com/geneasyura/cov19-hm/master/csv/maps.csv'

In [ ]:
req = tw.statuses.update(status=heads)
parent_tweet_id = req['id']
print("Tweeted({}):[{}]".format(parent_tweet_id, heads))

In [ ]:
f = codecs.open("diffs.txt", encoding="utf-8")
blk = ''
while True:
    l = f.readline()
    if not l:
        break
    #print(l)
    if len(blk + l) < 160:
        blk += l
    else:
        req = tw.statuses.update(status=blk, in_reply_to_status_id=parent_tweet_id)
        parent_tweet_id = req['id']
        print("Tweeted:[{}]".format(blk))
        blk = l
f.close()

In [ ]:
if len(blk) > 0:
    req = tw.statuses.update(status=blk, in_reply_to_status_id=parent_tweet_id)   
    print("Tweeted:[[{}]]".format(blk))

In [19]:
shutil.copyfile("maps.csv", "prev.csv")

'prev.csv'